# PROJECT BRIEF

Microsoft sees all the big companies creating original video content, and they want to get in on the fun. They have decided to create a new movie studio, but the problem is they don’t know anything about creating movies. They have hired you to help them better understand the movie industry.

Your team is charged with doing data analysis and creating a presentation that explores what type of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the CEO can use when deciding what type of films they should be creating.

# The Data

On the same MySQL instance from last week, in the schema `movies`, is some movie-related data from:
* Box Office Mojo
* IMDB
* Rotten Tomatoes
* TheMovieDB.org

# The Deliverables

There are three required deliverables for this project:

1. A well documented **Jupyter Notebook** containing any code you've written for this project and comments explaining it.
2. An organized **README.md** file in the GitHub repository that describes the contents of the repository. This file should be the source of information for navigating through the repository.
3. A Github repository containing all of the above files and any additional files worth includding
4. [OPTIONAL] A short **slide deck** (delivered as a PDF export) giving a high-level overview of your methodology and recommendations for non-technical stakeholders. If you do this, make sure to also add and commit this pdf of your non-technical presentation to your repository with a file name of presentation.pdf.

### Jupyter Notebook Must-Haves

For this project, your Jupyter Notebook should meet the following specifications:

#### Organization/Code Cleanliness

* The notebook should be well organized, easy to follow,  and code should be commented where appropriate.  
    * Level Up: The notebook contains well-formatted, professional looking markdown cells explaining any substantial code.  All functions have [docstrings](https://numpydoc.readthedocs.io/en/latest/format.html) that act as professional-quality documentation
* The notebook is written for technical audiences with a way to both understand your approach and reproduce your results. The target audience for this deliverable is other data scientists looking to validate your findings.

#### Visualizations & EDA

* Your project contains at least 4 meaningful data visualizations, with corresponding interpretations. All visualizations are well labeled with axes labels, a title, and a legend (when appropriate)  
* You pose at least 3 meaningful questions and answer them through EDA.  These questions should be well labeled and easy to identify inside the notebook.
    * **Level Up**: Each question is clearly answered with a visualization that makes the answer easy to understand.   
* Your notebook should contain 1 - 2 paragraphs briefly explaining your approach to this project.

# Our Questions To Answer

Beginning Question: Are the ratings consistent across sites?

1. How has the popularity (reviews/ratings) of certain genres changed over time?

2. Who are the most succesful (movie ratings) writers, directors and producers? 

3. Is there a correlation between production budget and domestic/worldwide earnings?

4. Is there an optimal run time and does that vary by genre?


In [110]:
#Import Packages and Connect To Database

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
%matplotlib inline

con = mysql.connector.connect(user='hbs-student', password='super-secure-password?',
                              host='hbs-ds-060120.c1doesqrid0e.us-east-1.rds.amazonaws.com',
                              database='movies')


In [37]:
# Get IMDB Tables

df_imdb_title_ratings= pd.read_sql_query("""
SELECT *
FROM imdb_title_ratings
;""", con)

df_imdb_title_ratings

,tconst,averagerating,numvotes
0,tt0063540,7.0,77
1,tt0066787,7.2,43
2,tt0069049,6.9,4517
3,tt0069204,6.1,13
4,tt0100275,6.5,119
...,...,...,...
73851,tt9913084,6.2,6
73852,tt9914286,8.7,136
73853,tt9914642,8.5,8
73854,tt9914942,6.6,5


In [3]:
# Get IMDB Tables

df_imdb_title_basics= pd.read_sql_query("""
SELECT *
FROM imdb_title_basics
;""", con)

df_imdb_title_basics

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,


In [32]:
#Merged IMDB Tables / Cleaned Run-Time

imdb_title_combined = pd.merge(df_imdb_title_basics, df_imdb_title_ratings, how='inner', on='tconst')

imdb_title_combined = imdb_title_combined.dropna(axis=0, subset=['runtime_minutes']).sort_values('start_year')


In [5]:
# Selecting Genres

imdb_title_combined[imdb_title_combined['genres'].str.contains('Drama')]

#OPEN QUESTION - HOW DO WE PHARSE OUT THE GENRES?

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
9065,tt1745772,My Heart Is an Idiot,My Heart Is an Idiot,2010,98.0,"Documentary,Drama,Romance",8.2,10
6181,tt1640402,Belukar,Belukar,2010,96.0,"Action,Drama,Romance",6.9,9
6173,tt1640198,Lost Years,Verloren jaren,2010,70.0,Drama,7.0,46
6166,tt1640111,Secret Love,Bimilae,2010,111.0,Drama,6.0,288
6165,tt1639901,A Screaming Man,Un homme qui crie,2010,92.0,Drama,6.7,1370
...,...,...,...,...,...,...,...,...
38657,tt3679070,Milan Talkies,Milan Talkies,2019,141.0,Drama,4.7,62
71417,tt8669356,Joan of Arc,Jeanne,2019,137.0,Drama,5.4,23
71312,tt8647376,Vai,Vai,2019,90.0,Drama,8.0,26
580,tt10052290,Love and Mercy,Milosc i milosierdzie,2019,104.0,"Biography,Drama,Family",5.4,18


In [33]:
# Converting RunTime to INT

imdb_title_combined['runtime_minutes'] = imdb_title_combined['runtime_minutes'].astype(int)
imdb_title_combined

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
9065,tt1745772,My Heart Is an Idiot,My Heart Is an Idiot,2010,98,"Documentary,Drama,Romance",8.2,10
6181,tt1640402,Belukar,Belukar,2010,96,"Action,Drama,Romance",6.9,9
6180,tt1640401,Andartu terlampau... 21 hari mencari suami,Andartu terlampau... 21 hari mencari suami,2010,90,Comedy,7.2,6
6177,tt1640220,Mr. Hush,Mr. Hush,2010,85,Horror,2.2,370
14040,tt1950461,Women of the Future,Women of the Future,2010,16,"Biography,Documentary",8.0,7
...,...,...,...,...,...,...,...,...
580,tt10052290,Love and Mercy,Milosc i milosierdzie,2019,104,"Biography,Drama,Family",5.4,18
582,tt10055770,Vellai Pookal,Vellaipookal,2019,122,Thriller,7.6,380
38919,tt3699702,The Divorce Party,The Divorce Party,2019,93,"Comedy,Romance",5.0,265
71311,tt8647310,Buffaloed,Buffaloed,2019,95,"Comedy,Drama",7.6,18


In [7]:
df_rt_movie_info= pd.read_sql_query("""
SELECT *
FROM rt_movie_info
;""", con)

df_rt_movie_info

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",,,104 minutes,None
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",,,116 minutes,None
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",,,128 minutes,None
4,7,,NR,Drama|Romance,Rodney Bennett,Giles Cooper,,,,,200 minutes,None
...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,,,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",,,88 minutes,Paramount Vantage
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,,"Jan 1, 1962","May 11, 2004",,,111 minutes,None
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",,,101 minutes,None


In [8]:
df_rt_reviews= pd.read_sql_query("""
SELECT *
FROM rt_movie_info
;""", con)

df_rt_reviews

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",,,104 minutes,None
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",,,116 minutes,None
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",,,128 minutes,None
4,7,,NR,Drama|Romance,Rodney Bennett,Giles Cooper,,,,,200 minutes,None
...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,,,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",,,88 minutes,Paramount Vantage
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,,"Jan 1, 1962","May 11, 2004",,,111 minutes,None
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",,,101 minutes,None


In [9]:
#Combining RT Tables

df_rt_combined = pd.merge(df_rt_movie_info,df_rt_reviews,how='inner',on='id')
df_rt_combined

,id,synopsis_x,rating_x,genre_x,director_x,writer_x,theater_date_x,dvd_date_x,currency_x,box_office_x,...,rating_y,genre_y,director_y,writer_y,theater_date_y,dvd_date_y,currency_y,box_office_y,runtime_y,studio_y
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",,,...,R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",,,104 minutes,None
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",...,R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",,,...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",,,116 minutes,None
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",,,...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",,,128 minutes,None
4,7,,NR,Drama|Romance,Rodney Bennett,Giles Cooper,,,,,...,NR,Drama|Romance,Rodney Bennett,Giles Cooper,,,,,200 minutes,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,,,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",...,R,Action and Adventure|Horror|Mystery and Suspense,,,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",,,...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",,,88 minutes,Paramount Vantage
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,,"Jan 1, 1962","May 11, 2004",,,...,G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,,"Jan 1, 1962","May 11, 2004",,,111 minutes,None
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",,,...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",,,101 minutes,None


In [10]:
#Clean RunTime Column

df_rt_combined['runtime'] = df_rt_combined['runtime_y'].str.split(" ").map(lambda x: x[0])

df_rt_combined['runtime'] = pd.to_numeric(df_rt_combined['runtime'])

df_rt_combined

,id,synopsis_x,rating_x,genre_x,director_x,writer_x,theater_date_x,dvd_date_x,currency_x,box_office_x,...,genre_y,director_y,writer_y,theater_date_y,dvd_date_y,currency_y,box_office_y,runtime_y,studio_y,runtime
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",,,...,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",,,104 minutes,None,104.0
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",...,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One,108.0
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",,,...,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",,,116 minutes,None,116.0
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",,,...,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",,,128 minutes,None,128.0
4,7,,NR,Drama|Romance,Rodney Bennett,Giles Cooper,,,,,...,Drama|Romance,Rodney Bennett,Giles Cooper,,,,,200 minutes,None,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,,,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",...,Action and Adventure|Horror|Mystery and Suspense,,,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema,106.0
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",,,...,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",,,88 minutes,Paramount Vantage,88.0
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,,"Jan 1, 1962","May 11, 2004",,,...,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,,"Jan 1, 1962","May 11, 2004",,,111 minutes,None,111.0
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",,,...,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",,,101 minutes,None,101.0


In [11]:
#Get TMDB Table

df_tmdb= pd.read_sql_query("""
SELECT *
FROM tmdb_movies
;""", con)

df_tmdb

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...
26512,26512,488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,26513,485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,26514,381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,26515,366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [12]:
#Convert Release_Date

df_tmdb['release_date'] = pd.to_datetime(df_tmdb['release_date'])

df_tmdb

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...
26512,26512,488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,26513,485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,26514,381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,26515,366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [13]:
#Get BOM Table

df_bom= pd.read_sql_query("""
SELECT *
FROM bom_movie_gross
;""", con)

df_bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3387 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   float64
 4   year            3387 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 132.4+ KB


In [14]:
#Create Worldwide Column

df_bom['foreign_gross'] = df_bom['foreign_gross'].fillna(0)

df_bom['worldwide_gross'] = df_bom['foreign_gross'] + df_bom['domestic_gross']
df_bom

,title,studio,domestic_gross,foreign_gross,year,worldwide_gross
0,Toy Story 3,BV,415000000.0,652000000.0,2010,1.067000e+09
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010,1.025500e+09
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010,9.603000e+08
3,Inception,WB,292600000.0,535700000.0,2010,8.283000e+08
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010,7.526000e+08
...,...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,0.0,2018,6.200000e+03
3383,Edward II (2018 re-release),FM,4800.0,0.0,2018,4.800000e+03
3384,El Pacto,Sony,2500.0,0.0,2018,2.500000e+03
3385,The Swan,Synergetic,2400.0,0.0,2018,2.400000e+03


In [15]:
#Get TN Table

df_tn= pd.read_sql_query("""
SELECT *
FROM tn_movie_budgets
;""", con)

df_tn

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [18]:
print(type(df_tn['production_budget'][0]))
print(type(df_tn['domestic_gross'][0]))
print(type(df_tn['worldwide_gross'][0]))

<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'str'>


In [21]:
#Clean Columns

df_tn['release_date'] = pd.to_datetime(df_tn['release_date'])
df_tn['production_budget'] = df_tn['production_budget'].str.split('$').map(lambda x: x[1].replace(',','')).astype(int)
df_tn['domestic_gross'] = df_tn['domestic_gross'].str.split('$').map(lambda x: x[1].replace(',','')).astype(int)
df_tn['worldwide_gross'] = df_tn['worldwide_gross'].str.split('$').map(lambda x: x[1].replace(',','')).astype(int)


# pd.to_numeric(df_tn['production_budget'])

AttributeError: Can only use .str accessor with string values!

In [46]:
#Add Foreign Column
pd.to_numeric(df_tn['worldwide_gross'])
df_tn['foreign_gross'] = df_tn['worldwide_gross'] - df_tn['domestic_gross']

df_tn.info()

ValueError: Unable to parse string "$2,776,345,279" at position 0

Queries from Josh start here

In [19]:
full_list = imdb_title_combined['genres'].str.split(',').apply(frozenset).to_frame(name='genres')
for genres in frozenset.union(*full_list.genres):
    full_list[genres] = full_list.apply(lambda _: int(genres in _.genres), axis=1)
full_list

,genres,,History,Action,News,Documentary,Thriller,Music,Comedy,Drama,...,Sport,War,Western,Crime,Reality-TV,Adventure,Mystery,Game-Show,Biography,Animation
9065,"(Romance, Drama, Documentary)",0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6181,"(Romance, Action, Drama)",0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6180,(Comedy),0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6177,(Horror),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14040,"(Biography, Documentary)",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,"(Family, Biography, Drama)",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
582,(Thriller),0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38919,"(Romance, Comedy)",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
71311,"(Drama, Comedy)",0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [30]:
df_imdb_tmdb= pd.read_sql_query("""
SELECT *
FROM tmdb_imdb_link
;""", con)
df_imdb_tmdb

,imdb_id,tmdb_id
0,1,16612
1,3,88013
2,7,159895
3,8,105158
4,10,774
...,...,...
62418,11057912,636593
62419,11101550,640427
62420,11108064,642749
62421,11168100,642203


In [34]:
#Clean IMDB IDs
imdb_title_combined['tconst'] = imdb_title_combined['tconst'].str.lstrip('tt')
imdb_title_combined = imdb_title_combined.rename(columns={'tconst': 'imdb_id'})
imdb_title_combined['imdb_id'] = imdb_title_combined['imdb_id'].astype(int)
#Clean TMDB IDs
df_tmdb = df_tmdb.rename(columns={'id': 'tmdb_id'})
#Merge
connected_imdb_tmdb = pd.merge(imdb_title_combined, df_imdb_tmdb, how='inner', on='imdb_id')
master_imdb_tmdb = pd.merge(connected_imdb_tmdb, df_tmdb, how='inner', on='tmdb_id')
master_imdb_tmdb = master_imdb_tmdb.rename(columns={'averagerating': 'averagerating_imdb', 'vote_average': 'vote_average_tmdb'})
master_imdb_tmdb

,imdb_id,primary_title,original_title_x,start_year,runtime_minutes,genres,averagerating_imdb,numvotes,tmdb_id,genre_ids,original_language,original_title_y,popularity,release_date,title,vote_average_tmdb,vote_count
0,1640220,Mr. Hush,Mr. Hush,2010,85,Horror,2.2,370,123389,849,en,Mr. Hush,2.042,2010-08-01,Mr. Hush,2.3,7
1,1639901,A Screaming Man,Un homme qui crie,2010,92,Drama,6.7,1370,57602,789,fr,Un homme qui crie,2.211,2010-09-28,A Screaming Man,6.1,10
2,1639426,Udaan,Udaan,2010,134,Drama,8.2,38827,45316,417,hi,उड़ान,5.376,2010-06-17,Udaan,7.2,83
3,1640680,Armadillo,Armadillo,2010,105,"Documentary,War",7.5,8906,37828,3023,da,Armadillo,4.631,2011-04-15,Armadillo,7.2,61
4,1641638,The Legend Is Born: Ip Man,Yip Man chin chyun,2010,100,"Action,Biography,Drama",7.1,29529,44249,928,cn,葉問前傳,1.832,2010-06-24,The Legend Is Born: Ip Man,6.7,319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10430,6921996,Johnny English Strikes Again,Johnny English Strikes Again,2018,89,"Action,Adventure,Comedy",6.2,46932,463272,23860,en,Johnny English Strikes Again,25.478,2018-10-26,Johnny English Strikes Again,6.2,1035
10431,2837574,The Old Man & the Gun,The Old Man & the Gun,2018,93,"Biography,Comedy,Crime",6.8,25902,429203,24004,en,The Old Man & the Gun,14.001,2018-09-28,The Old Man & the Gun,6.5,306
10432,5104604,Isle of Dogs,Isle of Dogs,2018,101,"Adventure,Animation,Comedy",7.9,108245,399174,23934,en,Isle of Dogs,16.954,2018-03-23,Isle of Dogs,7.9,1942
10433,6921496,A Bluebird in My Heart,A Bluebird in My Heart,2018,85,Drama,5.5,149,478308,25039,en,A Bluebird in My Heart,3.172,2018-03-10,A Bluebird in My Heart,7.6,9


In [47]:
df_imdb_crew = pd.read_sql_query("""
SELECT *
FROM imdb_title_crew
;""", con)
df_imdb_crew['tconst'] = df_imdb_crew['tconst'].str.lstrip('tt')
df_imdb_crew = df_imdb_crew.rename(columns={'tconst': 'imdb_id'})
df_imdb_crew['imdb_id'] = df_imdb_crew['imdb_id'].astype(int)

df_imdb_crew

,imdb_id,directors,writers
0,63540,nm0712540,"nm0023551,nm1194313,nm0347899,nm1391276"
1,66787,nm0002411,None
2,69049,nm0000080,"nm0000080,nm0462648"
3,69204,nm0611531,nm0347899
4,100275,"nm0765384,nm0749914","nm1360635,nm0749914"
...,...,...,...
146139,9916538,nm8185151,"nm4843252,nm4900525,nm2679404"
146140,9916622,"nm9272490,nm9272491","nm9272490,nm9272491"
146141,9916706,nm7764440,nm7933903
146142,9916730,nm10538612,nm10538612


In [48]:
connected_imdbcrew_tmdb = pd.merge(master_imdb_tmdb, df_imdb_crew, how='inner', on='imdb_id')


In [49]:
connected_imdbcrew_tmdb

,imdb_id,primary_title,original_title_x,start_year,runtime_minutes,genres,averagerating_imdb,numvotes,tmdb_id,genre_ids,original_language,original_title_y,popularity,release_date,title,vote_average_tmdb,vote_count,directors,writers
0,1640220,Mr. Hush,Mr. Hush,2010,85,Horror,2.2,370,123389,849,en,Mr. Hush,2.042,2010-08-01,Mr. Hush,2.3,7,nm3729006,nm3729006
1,1639901,A Screaming Man,Un homme qui crie,2010,92,Drama,6.7,1370,57602,789,fr,Un homme qui crie,2.211,2010-09-28,A Screaming Man,6.1,10,nm0757958,nm0757958
2,1639426,Udaan,Udaan,2010,134,Drama,8.2,38827,45316,417,hi,उड़ान,5.376,2010-06-17,Udaan,7.2,83,nm0003939,"nm0003939,nm0440604,nm4392112,nm4392385"
3,1640680,Armadillo,Armadillo,2010,105,"Documentary,War",7.5,8906,37828,3023,da,Armadillo,4.631,2011-04-15,Armadillo,7.2,61,nm2178458,nm1417214
4,1641638,The Legend Is Born: Ip Man,Yip Man chin chyun,2010,100,"Action,Biography,Drama",7.1,29529,44249,928,cn,葉問前傳,1.832,2010-06-24,The Legend Is Born: Ip Man,6.7,319,nm0946875,"nm0497231,nm0926075"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10430,6921996,Johnny English Strikes Again,Johnny English Strikes Again,2018,89,"Action,Adventure,Comedy",6.2,46932,463272,23860,en,Johnny English Strikes Again,25.478,2018-10-26,Johnny English Strikes Again,6.2,1035,nm1256143,nm0204030
10431,2837574,The Old Man & the Gun,The Old Man & the Gun,2018,93,"Biography,Comedy,Crime",6.8,25902,429203,24004,en,The Old Man & the Gun,14.001,2018-09-28,The Old Man & the Gun,6.5,306,nm1108007,"nm1108007,nm2970418"
10432,5104604,Isle of Dogs,Isle of Dogs,2018,101,"Adventure,Animation,Comedy",7.9,108245,399174,23934,en,Isle of Dogs,16.954,2018-03-23,Isle of Dogs,7.9,1942,nm0027572,"nm0027572,nm0178910,nm0005403,nm1440081"
10433,6921496,A Bluebird in My Heart,A Bluebird in My Heart,2018,85,Drama,5.5,149,478308,25039,en,A Bluebird in My Heart,3.172,2018-03-10,A Bluebird in My Heart,7.6,9,nm5747900,"nm5747900,nm9012185"


In [56]:
test = pd.read_sql_query("""
SELECT *
FROM imdb_title_principals
;""", con)

test.head()

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,,"[""The Man""]"
1,tt0111414,2,nm0398271,director,,
2,tt0111414,3,nm3739909,producer,producer,
3,tt0323808,10,nm0059247,editor,,
4,tt0323808,1,nm3579312,actress,,"[""Beth Boothby""]"


In [111]:
imdb_name = pd.read_sql_query("""
SELECT *
FROM imdb_name_basics
;""", con)

imdb_name.head(10)

OperationalError: MySQL Connection not available.

In [108]:
'actress' in imdb_name['primary_profession'][0]
#imdb_name['primary_profession'][0]

True

In [9]:
rt_movies = pd.read_sql_query("""
SELECT *
FROM rt_movie_info
;""", con)
rt_movies.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",,,104 minutes,None
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",,,116 minutes,None
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",,,128 minutes,None
4,7,,NR,Drama|Romance,Rodney Bennett,Giles Cooper,,,,,200 minutes,None


In [11]:
rt_reviews = pd.read_sql_query("""
SELECT *
FROM rt_reviews
;""", con)
rt_reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,,fresh,,0,Cinema Scope,"October 12, 2017"


In [27]:
rt_merged = pd.merge(rt_movies, rt_reviews, how = "inner",on = 'id')
rt_merged = rt_merged.rename(columns = {'rating_y':'user_ratings'})

In [101]:
rt_merged['user_ratings'].str.endswith('/4')

0        False
1        False
2        False
3        False
4        False
         ...  
12295    False
12296     True
12297     True
12298     True
12299    False
Name: user_ratings, Length: 12300, dtype: bool

In [100]:
#rt_user_ratings = rt_merged.user_ratings
indexNames_4 = rt_merged.loc[rt_merged['user_ratings'].str.endswith('/4')]['user_ratings']
indexNames_5 = rt_merged.loc[rt_merged['user_ratings'].str.endswith('/5')]['user_ratings']
indexNames_10 = rt_merged.loc[rt_merged['user_ratings'].str.endswith('/10')]['user_ratings']

indexNames_4=indexNames_4.str.replace("/4","").astype(float)*2.5
indexNames_5=indexNames_5.str.replace("/5","").astype(float)*2
indexNames_10=indexNames_10.str.replace("/10","").astype(float)

indexNames_4

12        5.00
14        7.50
18        5.00
19       10.00
21        2.50
         ...  
12290     8.75
12294     8.75
12296     8.75
12297     6.25
12298     8.75
Name: user_ratings, Length: 3016, dtype: float64